They have requested that you create a new table called online_transactions_fixed:
- (SQL) They realised the stock code field can’t be trusted so they would like you to join description to this table, without question marks. They also want to keep track of cases where description is missing for future analysis so use a left join.	
- (SQL) They would like you to remove all rows of data where customer_id is null
- (Python) They would like you to replace all missing values of Description with Unknown
- (Python) They would like you to remove any duplicated rows of data, and only keep the first appearance
- (SQL) They want you to remove the following stock codes
BANK CHARGES, POST, D, M, CRUK


In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect("../../week16/data/bootcamp_db")

In [3]:
query = """
select *
from online_transactions
limit 10
"""

test = pd.read_sql(query, conn)

In [4]:
test

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
0,536370,21791,24,2010-12-01 08:45:00,1.25,u12583,France
1,536373,82494L,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom
2,536378,21929,10,2010-12-01 09:37:00,1.95,u14688,United Kingdom
3,536381,37444A,1,2010-12-01 09:41:00,2.95,u15311,United Kingdom
4,536381,15056BL,2,2010-12-01 09:41:00,5.95,u15311,United Kingdom
5,536381,22438,1,2010-12-01 09:41:00,1.95,u15311,United Kingdom
6,536384,22469,40,2010-12-01 09:53:00,1.45,u18074,United Kingdom
7,536384,22189,4,2010-12-01 09:53:00,3.95,u18074,United Kingdom
8,536385,22783,1,2010-12-01 09:56:00,19.95,u1742,United Kingdom
9,536389,35004C,6,2010-12-01 10:03:00,5.45,u12431,Australia


In [5]:
query = """
select ot.*,
       sd.description
from online_transactions ot
/*joining the stock description table to the online transactions*/
left join (select *
           from stock_description
           where description <> '?') sd on ot.stock_code = sd.stock_code
/*remove rows of data where customer id is blank*/
where ot.customer_id <> ''
  /*remove rows of data where stock code is bank charges, post, d, m, cruk*/
  and ot.stock_code not in ('BANK CHARGES', 'POST', 'D', 'M', 'CRUK')
"""

In [6]:
ot_w_desc = pd.read_sql(query, conn)

In [7]:
ot_w_desc

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
0,536370,21791,24,2010-12-01 08:45:00,1.25,u12583,France,VINTAGE HEADS AND TAILS CARD GAME
1,536373,82494L,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,WOODEN FRAME ANTIQUE WHITE
2,536378,21929,10,2010-12-01 09:37:00,1.95,u14688,United Kingdom,JUMBO BAG PINK VINTAGE PAISLEY
3,536381,37444A,1,2010-12-01 09:41:00,2.95,u15311,United Kingdom,YELLOW BREAKFAST CUP AND SAUCER
4,536381,15056BL,2,2010-12-01 09:41:00,5.95,u15311,United Kingdom,EDWARDIAN PARASOL BLACK
...,...,...,...,...,...,...,...,...
405058,581580,22698,1,2011-12-09 12:20:00,2.95,u12748,United Kingdom,PINK REGENCY TEACUP AND SAUCER
405059,581584,20832,72,2011-12-09 12:25:00,0.72,u13777,United Kingdom,RED FLOCK LOVE HEART PHOTO FRAME
405060,581585,22178,12,2011-12-09 12:31:00,1.95,u15804,United Kingdom,VICTORIAN GLASS HANGING T-LIGHT
405061,581585,84692,25,2011-12-09 12:31:00,0.42,u15804,United Kingdom,BOX OF 24 COCKTAIL PARASOLS


In [8]:
ot_w_desc.shape

(405063, 8)

In [9]:
ot_w_desc.isnull().sum()

invoice            0
stock_code         0
quantity           0
invoice_date       0
price              0
customer_id        0
country            0
description     1175
dtype: int64

In [10]:
# replace missing description with unknown

ot_w_desc.description.fillna("UNKNOWN", inplace=True)

/var/folders/gs/c69h38_15hb4fnkpnc9gvksm0000gn/T/ipykernel_24507/3359609917.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ot_w_desc.description.fillna("UNKNOWN", inplace=True)


In [11]:
ot_w_desc.isnull().sum()

invoice         0
stock_code      0
quantity        0
invoice_date    0
price           0
customer_id     0
country         0
description     0
dtype: int64

In [12]:
ot_w_desc.description.value_counts().nlargest(20)

description
CREAM HANGING HEART T-LIGHT HOLDER    2077
REGENCY CAKESTAND 3 TIER              1905
JUMBO BAG RED RETROSPOT               1662
ASSORTED COLOUR BIRD ORNAMENT         1418
PARTY BUNTING                         1416
LUNCH BAG RED RETROSPOT               1359
SET OF 3 CAKE TINS PANTRY DESIGN      1232
UNKNOWN                               1175
LUNCH BAG  BLACK SKULL.               1126
POPCORN HOLDER                        1118
JUMBO BAG VINTAGE DOILEY              1115
LUNCH BAG SUKI DESIGN                 1103
PACK OF 72 RETROSPOT CAKE CASES       1080
LUNCH BAG VINTAGE DOILEY              1040
BUNTING , SPOTTY                      1036
PAPER CHAIN KIT 50'S CHRISTMAS        1029
LUNCH BAG SPACEBOY DESIGN             1021
LUNCH BAG CARS BLUE                   1012
NATURAL SLATE HEART CHALKBOARD         997
HEART OF WICKER SMALL                  996
Name: count, dtype: int64

In [13]:
# drop the duplicate rows of data, keeping the first appearance only

ot_cleaned = ot_w_desc.drop_duplicates(keep="first")

In [17]:
ot_cleaned.shape

(399841, 8)

In [18]:
# store as a .csv file to local data folder

ot_cleaned.to_csv("../data/ot_final.csv", index=False)

In [20]:
# check

check = pd.read_csv("../data/ot_final.csv")
check.head()

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
0,536370,21791,24,2010-12-01 08:45:00,1.25,u12583,France,VINTAGE HEADS AND TAILS CARD GAME
1,536373,82494L,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,WOODEN FRAME ANTIQUE WHITE
2,536378,21929,10,2010-12-01 09:37:00,1.95,u14688,United Kingdom,JUMBO BAG PINK VINTAGE PAISLEY
3,536381,37444A,1,2010-12-01 09:41:00,2.95,u15311,United Kingdom,YELLOW BREAKFAST CUP AND SAUCER
4,536381,15056BL,2,2010-12-01 09:41:00,5.95,u15311,United Kingdom,EDWARDIAN PARASOL BLACK


In [21]:
check.shape

(399841, 8)